<a href="https://colab.research.google.com/github/haitam-zouhair/workspace_henceforth/blob/master/arabic_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%%shell
cp -r 'drive/My Drive/Projects/Zouhair/' .

In [ ]:
import os
os.chdir("Zouhair")

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 13.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=424726c3228633d827ba9836be34b398d0dcb82ad43896bb9faea81a3ae19f3d
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=f7ed8662c293de6eee0c2bd1ee1b2b6b9c77816883746c1d8e513eb891307f15
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


# Dataset

I used [SANAD](https://www.sciencedirect.com/science/article/pii/S2352340919304305) as dataset. Dataset can be downloaded [here](https://data.mendeley.com/datasets/57zpx667y9/2), you can also use directly the dataset from Google Drive, I put them [here](https://drive.google.com/file/d/1hjDR0C5vHPckZisqYby13r2m1i5mtH3p/view?usp=sharing).

In [ ]:
%%shell
unzip arabic.zip
unzip Akhbarona.zip -d Akhbarona
unzip Arabiya.zip -d Arabiya
unzip Khaleej.zip -d Khaleej
unzip SANAD_SUBSET.zip -d SANAD_SUBSET
rm *.zip 

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: SANAD_SUBSET/khaleej/Train/Culture/0f8c6c1fbfb1.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2279dc2b4ef0.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/34c8bdda56b8.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2a1dd79f213f.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/06e7a2f41572.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2480530a7c95.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/12ed0be04318.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/37cee96d9287.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2a4af8ef77b8.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/14bea0b04409.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2af68aa24de2.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/23c87582fe37.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/279386f5a587.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/178

In [ ]:
import pandas as pd


DATANAME = "Arabiya"

data_frame = pd.DataFrame(columns=["text", "label"])

for cat in os.listdir(DATANAME):
    print("reading files from category {}...".format(cat))
    dir_path = os.path.join(DATANAME, cat)

    for file_name in os.listdir(dir_path):
        path = os.path.join(dir_path, file_name)
        
        with open(path, "r") as f:
            text = f.read()

        data_frame = data_frame.append({"text": text, "label": cat},
                                       ignore_index=True)


reading files from category Finance...
reading files from category Sports...
reading files from category Politics...
reading files from category Medical...
reading files from category Religion...
reading files from category Tech...
reading files from category Culture...


In [ ]:
print(len(data_frame))
data_frame.head()

71246


,text,label
0,سجلت قبرص الجزيرة المتوسطية أفضل رقم في عدد ا...,Finance
1,كشف رئيس لجنة المقاولات بغرفة الشرقية عبدالحكي...,Finance
2,أكد البنك الدولي توقف إنتاج النفط والغاز في ال...,Finance
3,قال بنك تونس والإمارات إن تونس وجهاز أبوظبي لل...,Finance
4,ارتفع إقراض البنوك في الإمارات بواقع 8% على أس...,Finance


# I/-Arabic Text Processing

Similar to English text, arabic text should be preporcessed before performing any NLP task on it. The pre-processing steps are roughly the same:

* **Tokenization:**
* **Remove Stopwords:** Arabic stop words can be found [here](https://github.com/mohataher/arabic-stop-words/blob/master/list.txt).
* **Lemmatization**:  I am not sure how to lemmatize arabic text, I think in arabic steeming and lemmatization are the same text. I have found this [paper](https://www.aclweb.org/anthology/L18-1181.pdf)  but we need to check this later to be sure (**TO DO**).
* **Stemming**: We use `nltk.SRIStemmer`



In [ ]:
%%shell
#Download arabic stop words
wget https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt
mv list.txt arabic_stop_words.txt

--2020-06-24 21:12:42--  https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6474 (6.3K) [text/plain]
Saving to: ‘list.txt’

list.txt            100%[===================>]   6.32K  --.-KB/s    in 0s      

2020-06-24 21:12:42 (76.6 MB/s) - ‘list.txt’ saved [6474/6474]



In [ ]:
from string import punctuation
from nltk import ISRIStemmer


punctuation += '،؛؟”0123456789“'
stop_words = open("arabic_stop_words.txt").read().splitlines()
stemmer = ISRIStemmer()

def preprocess(text):
    # tokenize
    text = ''.join(c for c in text if c not in punctuation)
    tokens = text.split()  

    # remove stop words   
    tokens = [w for w in tokens if w not in stop_words]

    # stem
    stems = [stemmer.stem(w) for w in tokens]

    return stems


processed_docs = data_frame['text'].map(preprocess)
data_frame['processed_text'] = processed_docs

# II/- Topic modeling using LDA


In [ ]:
from gensim import corpora, models

import numpy as np

## 1. Bag of Words

In [ ]:
# build bag of words dictionnary
dictionary = corpora.Dictionary(data_frame['processed_text'])

# filter iinfrequent words
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data_frame['processed_text']]

View Bag of Word result

In [ ]:
for i in range(len(bow_corpus[0])):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[0][i][0], 
                                               dictionary[bow_corpus[0][i][0]], 
                                               bow_corpus[0][i][1]))

Word 0 ("أضف") appears 1 time.
Word 1 ("أظهر") appears 1 time.
Word 2 ("ؤشر") appears 1 time.
Word 3 ("ارخ") appears 1 time.
Word 4 ("اضة") appears 1 time.
Word 5 ("اقتصادية") appears 1 time.
Word 6 ("الة") appears 1 time.
Word 7 ("الي") appears 1 time.
Word 8 ("امن") appears 2 time.
Word 9 ("برط") appears 3 time.
Word 10 ("بلغ") appears 4 time.
Word 11 ("بين") appears 3 time.
Word 12 ("تحس") appears 1 time.
Word 13 ("تعثر") appears 1 time.
Word 14 ("تعد") appears 3 time.
Word 15 ("توسط") appears 3 time.
Word 16 ("ثاب") appears 1 time.
Word 17 ("جدد") appears 1 time.
Word 18 ("جزر") appears 4 time.
Word 19 ("جمل") appears 1 time.
Word 20 ("حدد") appears 1 time.
Word 21 ("حدى") appears 1 time.
Word 22 ("حسب") appears 1 time.
Word 23 ("حلي") appears 2 time.
Word 24 ("حينذ") appears 1 time.
Word 25 ("خرى") appears 2 time.
Word 26 ("خطة") appears 1 time.
Word 27 ("خمس") appears 1 time.
Word 28 ("خير") appears 1 time.
Word 29 ("دخل") appears 1 time.
Word 30 ("دعم") appears 1 time.
Word 31 (

## 2. Running LDA

Our dataset has $6$ topics, we will try to build LDA model with $6$ topics.

In [ ]:
NUM_TOPICS = 6

lda_model = models.LdaMulticore(bow_corpus,
                                num_topics=NUM_TOPICS,
                                id2word=dictionary,
                                passes=2,
                                workers=4)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} | Word: {}'.format(idx, topic))

Topic: 0 | Word: 0.014*"رفع" + 0.014*"شرك" + 0.012*"سوق" + 0.012*"عمل" + 0.012*"سعر" + 0.011*"عام" + 0.010*"نسب" + 0.009*"نفط" + 0.009*"دول" + 0.009*"بنك"
Topic: 1 | Word: 0.021*"لعب" + 0.018*"فرق" + 0.013*"دور" + 0.011*"هدف" + 0.011*"برا" + 0.009*"قدم" + 0.009*"ندي" + 0.008*"بطل" + 0.008*"درب" + 0.007*"نخب"
Topic: 2 | Word: 0.009*"عمل" + 0.007*"علم" + 0.007*"عرض" + 0.007*"حمد" + 0.006*"جمع" + 0.006*"فنن" + 0.006*"سعد" + 0.006*"عرب" + 0.006*"قدم" + 0.006*"امر"
Topic: 3 | Word: 0.013*"خدم" + 0.010*"عمل" + 0.010*"شرك" + 0.008*"هتف" + 0.008*"علم" + 0.007*"طبق" + 0.007*"جدد" + 0.007*"جهز" + 0.006*"كثر" + 0.006*"جمع"
Topic: 4 | Word: 0.010*"نطق" + 0.010*"نقط" + 0.010*"سور" + 0.009*"حكم" + 0.008*"عرق" + 0.007*"هدف" + 0.007*"ركز" + 0.007*"صدر" + 0.007*"نظم" + 0.007*"عمل"
Topic: 5 | Word: 0.016*"شرك" + 0.015*"عمل" + 0.011*"سعد" + 0.009*"جمع" + 0.008*"علم" + 0.006*"شرع" + 0.006*"خدم" + 0.006*"قدم" + 0.006*"دول" + 0.006*"وزر"


Seems that we can recognize some topics from those results

* Topic 0 corresponds to **Finance**
* Topic 1 corresponds to **Sports**
* Topic 2 corresponds to **Culture**
* Topic 3 corresponds to **Tech**   
* Topic 4 correnponds to **Finance**
* Topic 5 corresponds to **Politics**




# III/- Evaluation

## 1) Vizualisation

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.144088 -0.017362       1        1  27.224085
0     -0.115744 -0.072045       2        1  22.056501
5     -0.052729  0.020748       3        1  17.784634
2      0.044014  0.047369       4        1  14.371693
3     -0.044766  0.105336       5        1  12.382441
4      0.025136 -0.084045       6        1   6.180643, topic_info=     Term          Freq         Total Category  logprob  loglift
559   لعب  74720.000000  74720.000000  Default  30.0000  30.0000
357   فرق  71395.000000  71395.000000  Default  29.0000  29.0000
1660  برا  38812.000000  38812.000000  Default  28.0000  28.0000
254   نقط  27889.000000  27889.000000  Default  27.0000  27.0000
223   خدم  40001.000000  40001.000000  Default  26.0000  26.0000
...   ...           ...           ...      ...      ...      ...
142   دول   3615.004150  55791.230469   Topic6  -5.3023   0.0472
310   تحد   3377.490479  46375.304688   Topic6  -5.3702   0.1641
100   وزر   2810.489502  23867.955078   Topic6  -5.5540   0.6446
177   قدم   3296.355469  66776.468750   Topic6  -5.3946  -0.2248
1230  دفع   2712.405762  29403.021484   Topic6  -5.5895   0.4005

[593 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
9540      5  0.989210      AMOLED
9942      5  0.972024  BlackBerry
9923      3  0.958157        Book
9923      5  0.018078        Book
5529      1  0.036787         EGX
...     ...       ...         ...
5223      5  0.006725           ٫
5223      6  0.006725           ٫
8354      2  0.006707           ″
8354      3  0.006707           ″
8354      5  0.985880           ″

[2433 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 6, 3, 4, 5])

## 2. Perplexity and Coherence Score

In [ ]:
coherence_model_lda = models.CoherenceModel(model=lda_model,
                                     texts=data_frame["processed_text"],
                                     dictionary=dictionary,
                                     coherence='c_v')

print('Coherence Score: {:.2f}'.format(coherence_model_lda.get_coherence()))

Coherence Score: 0.43


# IV/- Content classification using topic modeling

In this section, we use the results of topic modeling in order to build a content classifier. Each document will be represented by a `NUM_TOPICS`-vector capturing the distribution of the learnt topics. This is similar to the approach used in [Phan et al. (2008)](http://gibbslda.sourceforge.net/fp224-phan.pdf)

## 1. Build dataset

In [ ]:
N = len(bow_corpus)


X = np.zeros((N, NUM_TOPICS))
y = np.zeros(N, dtype=np.uint8)


label2id = {label: idx for (idx, label)
 in enumerate(list(set(data_frame["label"])))}

for i in range(N):
    topics_proba = lda_model.get_document_topics(bow_corpus[i],
                                              minimum_probability=0.0)
    
    topic_probas = [v for _, v in  topics_proba] 
    X[i,:] = np.array(topic_probas)
    y[i] = label2id[data_frame["label"][i]]
    

## 2. Train classifier

We train some simple classifiers using the extracted feature vectors

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# SVM
svm_clf = SVC()
svm_clf.fit(X_train, y_train)

train_acc = svm_clf.score(X_train, y_train)
test_acc = svm_clf.score(X_test, y_test)
print("SVM: Train accuracy {0:.2f} | Test acuracy {1:.2f}".format(train_acc,
                                                                  test_acc))

SVM: Train accuracy 0.92 | Test acuracy 0.92


# V/- Model Deployment

In this section we save the model, and we load it and test it on new text



In [ ]:
model_path = "model/model"

lda_model.save(model_path)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
